In [1]:
import json
from my_utils import NERDataset,collate_fn
from torch.utils.data import Dataset,DataLoader

In [2]:
dataset = NERDataset('nlp2024-data/dataset/dev.json')
dataloader = DataLoader(dataset,batch_size=8,collate_fn=collate_fn)
for batch in dataloader:
    print(batch)
    break

([[2584, 825, 2075, 1147, 1814, 1019, 122, 1515, 1147, 2448, 737, 2670, 926, 2448, 737, 2670, 1983, 970, 899, 564, 2075, 899, 1518, 737], [1147, 1814, 2670, 570, 2501, 2485, 737, '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#'], [2670, 2738, 1123, 2696, 2448, 737, '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#'], [2670, 2959, 2501, '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#'], [1983, 2738, 870, 2075, 864, 1983, 926, 899, 1518, '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#'], [2747, 1542, 2670, 2093, 210, 2448, 737, '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#'], [2670, 1983, 1549, 2604, 2696, 1455, 737, '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#'], [1983, 2670, '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#'

In [20]:
## 获取所有字符
data_dev = json.load(open('nlp2024-data/dataset/dev.json',encoding='utf-8'))
data_train = json.load(open('nlp2024-data/dataset/train.json',encoding='utf-8'))
def get_text_set(data):
    text_set = set()
    for key,val in data.items():
        for character in val['diagnosis']:
            text_set.add(character)
        for character in val['self_report']:
            text_set.add(character)
        dialogues = val['dialogue']
        for dia in dialogues:
            for character in dia['speaker']:
                text_set.add(character)
            for character in dia['sentence']:
                text_set.add(character)
            for word in dia['symptom_norm']:
                for character in word:
                    text_set.add(character)
    return text_set
text_set_train = get_text_set(data_train)
text_set_dev = get_text_set(data_dev)
text_set = text_set_dev | text_set_train
len(text_set)
text_list = list(text_set)
text_list.append('#')
text_dict = {text_list[i]:i for i in range(len(text_list))}
with open('char2id.json','w',encoding='utf-8') as f:
    json.dump(text_dict,f,ensure_ascii=False)

In [2]:
from transformers import BertModel,BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertModel.from_pretrained('bert-base-chinese')

/Users/lijinliang/opt/anaconda3/envs/pytorch/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/lijinliang/opt/anaconda3/envs/pytorch/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
text = '你好'
inputs = tokenizer(text,return_tensors='pt',padding=True,truncation=True)
inputs

{'input_ids': tensor([[ 101,  872, 1962,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [13]:
out = model(**inputs)

In [16]:
out.last_hidden_state.shape

torch.Size([1, 4, 768])

In [3]:
from my_utils import NERDatasetBert, collate_fn_bert
datasetBert = NERDatasetBert('nlp2024-data/dataset/dev.json',tokenizer)
dataloader = DataLoader(datasetBert,batch_size=2, collate_fn=collate_fn_bert)


In [5]:
for batch in dataloader:
    inputs = {"input_ids":batch[0],
              "attention_mask":batch[1]}
    out = model(**inputs)
    print(out)
    break

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.7955, -0.0604, -0.5145,  ..., -0.0065, -0.0573,  0.0602],
         [ 0.0069, -0.1325,  0.1473,  ..., -0.6525, -0.4242,  0.1753],
         [ 0.8572,  0.2717,  0.3732,  ..., -0.1347,  0.5311, -0.4706],
         ...,
         [ 0.6514, -0.5964, -0.2139,  ...,  0.6149,  0.1953,  0.6304],
         [ 0.5118,  0.4172, -0.5241,  ..., -0.5596,  0.4079, -0.0406],
         [ 0.0385, -0.4536, -0.0262,  ..., -0.3393, -0.2469,  0.2462]],

        [[ 0.4655, -0.4097, -0.0354,  ...,  0.1277,  0.0083, -0.1648],
         [ 0.0356, -0.2946, -0.5383,  ..., -0.3654, -0.5864,  0.3523],
         [ 0.5439, -1.2687, -0.8399,  ...,  0.5940, -0.0624,  0.2715],
         ...,
         [ 0.2470, -0.0931, -0.1315,  ..., -0.3311, -0.1926, -0.1254],
         [ 0.1547, -0.4363, -0.2219,  ..., -0.1255, -0.2254, -0.0903],
         [ 0.1825, -0.3251, -0.1521,  ..., -0.1516, -0.2679, -0.1133]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_ou

In [2]:
import pandas as pd
data = pd.read_csv('nlp2024-data/dataset/symptom_norm.csv')
data.to_dict()

{'norm': {0: '咳嗽',
  1: '发热',
  2: '感冒',
  3: '鼻流涕',
  4: '腹泻',
  5: '痰',
  6: '呕吐',
  7: '中等度热',
  8: '稀便',
  9: '高热',
  10: '消化不良',
  11: '肺炎',
  12: '支气管炎',
  13: '鼻塞',
  14: '病毒感染',
  15: '低热',
  16: '黄疸',
  17: '细菌感染',
  18: '哭闹',
  19: '喷嚏',
  20: '便秘',
  21: '退热',
  22: '呼吸道感染',
  23: '腹胀',
  24: '喘息',
  25: '脱水',
  26: '干咳',
  27: '过敏',
  28: '咽喉炎',
  29: '绿便',
  30: '精神软',
  31: '皮疹',
  32: '水样便',
  33: '出汗',
  34: '屁',
  35: '上呼吸道感染',
  36: '支气管肺炎',
  37: '肠炎',
  38: '支原体感染',
  39: '腹痛',
  40: '嗓子沙哑',
  41: '大便粘液',
  42: '痰鸣音',
  43: '食欲不振',
  44: '炎症',
  45: '恶心',
  46: '气喘',
  47: '气管炎',
  48: '咳痰',
  49: '干呕',
  50: '扁桃体炎',
  51: '幼儿急疹',
  52: '嗜睡',
  53: '湿疹',
  54: '尿量减少',
  55: '上火',
  56: '脓血便',
  57: '抽搐',
  58: '胃肠功能紊乱',
  59: '四肢厥冷',
  60: '肠鸣音',
  61: '疱疹',
  62: '头痛',
  63: '鼻炎',
  64: '缺钙',
  65: '贫血',
  66: '血便',
  67: '过敏体质',
  68: '咽喉不适',
  69: '过敏性咳嗽',
  70: '大便酸臭',
  71: '呼吸困难',
  72: '电解质紊乱',
  73: '病理性黄疸',
  74: '口腔溃疡',
  75: '蛋花汤样便',
  76: '烦躁',
  77: '充血